In [1]:
import tensorflow as tf
import numpy as np
import os
import time
import datetime
import itertools
from collections import Counter
import data_helpers

In [3]:
x, y, vocabulary, vocabulary_inv = data_helpers.load_data()

In [5]:
np.random.seed(10)
# Randomly shuffle data
shuffle_indices = np.random.permutation(np.arange(len(y)))
x_shuffled = x[shuffle_indices]
y_shuffled = y[shuffle_indices]

# Split train/test set
# TODO: This is very crude, should use cross-validation
x_train, x_dev = x_shuffled[:-1000], x_shuffled[-1000:]
y_train, y_dev = y_shuffled[:-1000], y_shuffled[-1000:]

In [6]:
class TextCNN(object):
    """
    A CNN for text classification.
    """
    def __init__(
        self, vocabulary_size, sequence_length, num_classes=2, embedding_size=128,
        filter_sizes=[3, 4, 5], num_filters=100):
        
        # Placeholders for input and output and dropout
        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")
        
        # Embedding layer
        with tf.device('/cpu:0'), tf.name_scope("embedding"):
            W = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0), name="W")
            self.embedded_chars = tf.nn.embedding_lookup(W, self.input_x)
        self.embedded_chars_expanded = tf.expand_dims(self.embedded_chars, -1)

        # Create a convolution + maxpool layer for each filter size
        pooled_outputs = []
        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope("conv-maxpool-%s" % filter_size):
                # Convolution Layer
                filter_shape = [filter_size, embedding_size, 1, num_filters]
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
                conv = tf.nn.conv2d(
                    self.embedded_chars_expanded,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                # Maxpooling over the outputs
                pooled = tf.nn.max_pool(
                    h,
                    ksize=[1, sequence_length - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                pooled_outputs.append(pooled)

        # Combine all the pooled features
        num_filters_total = num_filters * len(filter_sizes)
        self.h_pool = tf.concat(3, pooled_outputs)
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])
        
        # Add dropout
        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)
        
        # Final (unnormalized) scores and predictions
        with tf.name_scope("output"):
            W = tf.Variable(tf.truncated_normal([num_filters_total, num_classes], stddev=0.1), name="W")
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]))
            self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="scores")
            self.predictions = tf.argmax(self.scores, 1, name="predictions")

        # Mean cross-entropy loss
        with tf.name_scope("loss"):
            losses = tf.nn.softmax_cross_entropy_with_logits(self.scores, self.input_y)
            self.loss = tf.reduce_mean(losses)
        
        # Expression for the accuracy
        with tf.name_scope("accuracy"):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")
        

In [9]:
BATCH_SIZE = 64
NUM_EPOCHS = 500
EVALUATE_EVERY = CHECKPOINT_EVERY = 100
DROPOUT_KEEP_PROB = 0.5
SEQUENCE_LENGTH = x_train.shape[1]

with tf.Graph().as_default():
    session_conf = tf.ConfigProto(allow_soft_placement=True)      
    sess = tf.Session(config=session_conf)  
    with sess.as_default():
        cnn = TextCNN(
            vocabulary_size=len(vocabulary),
            sequence_length=SEQUENCE_LENGTH,
            num_classes=2,
            embedding_size=128,
            filter_sizes=[3, 4, 5],
            num_filters=80)
        
        # Define Training procedure
        global_step = tf.Variable(0, name="global_step")
        optimizer = tf.train.AdamOptimizer(1e-4)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)
        
        # Keep track of gradient values and sparsity
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.histogram_summary("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.scalar_summary("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.merge_summary(grad_summaries)
        
        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))
        
        # Train Summaries
        loss_summary = tf.scalar_summary("loss", cnn.loss)
        acc_summary = tf.scalar_summary("accuracy", cnn.accuracy)
        train_summary_op = tf.merge_summary([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.train.SummaryWriter(train_summary_dir, sess.graph_def)

        # Dev summaries
        dev_summary_op = tf.merge_summary([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.train.SummaryWriter(dev_summary_dir, sess.graph_def)
       
        # Checkpointing
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        # Tensorflow assumes this directory already exists so we need to create it
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.all_variables())
        
        # Initialize all variables
        sess.run(tf.initialize_all_variables())
    
        # A single training step
        def train_step(x_batch, y_batch):
            feed_dict = { cnn.input_x: x_batch, cnn.input_y: y_batch, cnn.dropout_keep_prob: DROPOUT_KEEP_PROB }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)
            
        def dev_step(x_batch, y_batch, writer=None):
            feed_dict = {cnn.input_x: x_batch, cnn.input_y: y_batch, cnn.dropout_keep_prob: 1.0 }
            step, summaries, loss, accuracy = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)
        
        batches = data_helpers.batch_iter(zip(x_train, y_train), BATCH_SIZE, NUM_EPOCHS)
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % EVALUATE_EVERY == 0:
                print("\nDev Set:")
                dev_step(x_dev, y_dev, writer=dev_summary_writer)
                print("")
            if current_step % CHECKPOINT_EVERY == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))                
        
        

Writing to /Users/dennybritz/projects/wildml/cnn-text-classification-tf/runs/1449731188

2015-12-10T08:06:29.306352: step 1, loss 2.46018, acc 0.453125
2015-12-10T08:06:29.598443: step 2, loss 2.21895, acc 0.5
2015-12-10T08:06:29.860732: step 3, loss 2.13484, acc 0.484375
2015-12-10T08:06:30.127405: step 4, loss 1.67236, acc 0.53125
2015-12-10T08:06:30.365780: step 5, loss 1.8633, acc 0.46875
2015-12-10T08:06:30.591764: step 6, loss 2.51452, acc 0.4375
2015-12-10T08:06:30.820956: step 7, loss 1.81396, acc 0.546875
2015-12-10T08:06:31.047919: step 8, loss 1.48453, acc 0.625
2015-12-10T08:06:31.271223: step 9, loss 1.64171, acc 0.515625
2015-12-10T08:06:31.490915: step 10, loss 2.04186, acc 0.484375
2015-12-10T08:06:31.719753: step 11, loss 1.545, acc 0.5625
2015-12-10T08:06:31.947065: step 12, loss 1.76173, acc 0.5
2015-12-10T08:06:32.184173: step 13, loss 2.009, acc 0.5625
2015-12-10T08:06:32.678610: step 14, loss 1.74997, acc 0.5
2015-12-10T08:06:32.898872: step 15, loss 1.85376, acc 

KeyboardInterrupt: 